In [1]:
import pandas as pd
import torch
from transformers import BertTokenizer

# Load Combined Dataset (with both fake and true news)
df = pd.read_csv("news.csv")

# Ensure necessary columns exist
df = df[['title', 'text', 'False/Truth']]  # 'fake/true' is the label column (0 = fake, 1 = true)

# Handle missing values if any
df['title'] = df['title'].fillna('missing')
df['text'] = df['text'].fillna('missing')

# Map the 'fake/true' column to numeric labels (0 = fake, 1 = true)
df['label'] = df['False/Truth']

# Combine 'title' and 'text' into one input string
df['combined_text'] = df['title'] + " " + df['text']

# Load Tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the combined text
def tokenize_text(text):
    return tokenizer(text, padding="max_length", truncation=True, max_length=512)

df['tokens'] = df['combined_text'].apply(tokenize_text)

# Convert tokenized outputs to tensors
input_ids = torch.tensor([x['input_ids'] for x in df['tokens']])
attention_masks = torch.tensor([x['attention_mask'] for x in df['tokens']])
labels = torch.tensor(df['label'].values).long()  # Ensure labels are integers

# Save processed data
torch.save((input_ids, attention_masks, labels), "processed_combined_data.pt")

print("✅ Combined Data processed and saved successfully!")



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

✅ Combined Data processed and saved successfully!


In [2]:
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer

# Define a custom dataset class
class NewsDataset(Dataset):
    def __init__(self, input_ids, attention_masks, labels):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_masks[idx],
            'labels': self.labels[idx]
        }

# Load your processed data
input_ids, attention_masks, labels = torch.load("processed_combined_data.pt")

# Split the data into train and validation sets
train_size = int(0.8 * len(input_ids))
val_size = len(input_ids) - train_size

train_dataset = NewsDataset(input_ids[:train_size], attention_masks[:train_size], labels[:train_size])
val_dataset = NewsDataset(input_ids[train_size:], attention_masks[train_size:], labels[train_size:])

# Define the model
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Define training arguments
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8
)

# Train the model using the Trainer
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()

# Save the model
model.save_pretrained("./bert_fake_true_news_model")
shutil.make_archive("bert_fake_news_model", 'zip', "./bert_fake_news_model")
files.download("bert_fake_news_model.zip")

print("✅ Model trained and saved!")


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: eliasxmas (eliasxmas-hogeschool-ucll) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.042900,0.005791
2,0.000000,0.002373
3,0.000000,0.002604


NameError: name 'shutil' is not defined

In [6]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")  # Use the same tokenizer as training

# Load Model
model = AutoModelForSequenceClassification.from_pretrained("./bert_fake_true_news_model", trust_remote_code=True)

# Example Prediction
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    prediction = torch.argmax(logits, dim=1).item()
    return "Fake News" if prediction == 0 else "True News"

# Test with an example
print(predict("people are born as babies"))


Fake News


In [12]:
#@title save yo data to drive
filename = "content" #@param {type:"string"}
folders_or_files_to_save = "/content" #@param {type:"string"}
from google.colab import files
from google.colab import auth
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build

def save_file_to_drive(name, path):
    file_metadata = {
    'name': name,
    'mimeType': 'application/octet-stream'
    }

    media = MediaFileUpload(path,
                  mimetype='application/octet-stream',
                  resumable=True)

    created = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()

    print('File ID: {}'.format(created.get('id')))

    return created


extension_zip = ".zip"

zip_file = filename + extension_zip

# !rm -rf $zip_file
!zip -r $zip_file {folders_or_files_to_save} # FOLDERS TO SAVE INTO ZIP FILE

auth.authenticate_user()
drive_service = build('drive', 'v3')

destination_name = zip_file
path_to_file = zip_file
save_file_to_drive(destination_name, path_to_file)

  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/default_configs.db (deflated 98%)
  adding: content/.config/hidden_gcloud_config_universe_descriptor_data_cache_configs.db (deflated 97%)
  adding: content/.config/configurations/ (stored 0%)
  adding: content/.config/configurations/config_default (stored 0%)
  adding: content/.config/.last_opt_in_prompt.yaml (stored 0%)
  adding: content/.config/.last_survey_prompt.yaml (stored 0%)
  adding: content/.config/active_config (stored 0%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2025.03.17/ (stored 0%)
  adding: content/.config/logs/2025.03.17/13.31.50.595714.log (deflated 58%)
  adding: content/.config/logs/2025.03.17/13.31.59.213557.log (deflated 57%)
  adding: content/.config/logs/2025.03.17/13.31.45.377163.log (deflated 87%)
  adding: content/.config/logs/2025.03.17/13.31.05.585644.log (deflated 92%)
  adding: content/.config/logs/2025.03.17/13.31.36.425885.l

{'id': '1auuLQ3TYxRpUlL_ab7EQsvefNKT6Oqc4'}